# Segment 4 Lab 1

## Introduction to this segment's labs

# Introducing: The Price Is Right

We are going to build a multi-agent framework that will:
1. Pull RSS feeds from the web and interpret them, looking for promising deals (multi-shot prompting, structured outputs)
2. Use RAG and a Frontier Model to estimate the price of a product, using a knowledge base
3. Use a QLoRA fine tuned LLM to estimate the price
4. Send a push notification when it finds a deal that's worth significantly less than it estimates
5. A planning agent will orchestrate this

### We'll be going through this fast - the idea is for you to come back and try it yourself!


## First: Let's look at the Agent code for interpreting RSS feeds using multi-shot prompting

In [1]:
# Some imports

from dotenv import load_dotenv
from agents.deals import ScrapedDeal
from openai import OpenAI
import logging

In [2]:
# Set up our env variables as before

load_dotenv(override=True)
openai = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# I wrote this code to fetch RSS feeds

scraped_deals = ScrapedDeal.fetch(show_progress=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.38s/it]


In [5]:
print(scraped_deals[2].describe())

Title: Certified Refurb Sony WH-1000XM5 Wireless Headphones for $180 + free shipping
Details: It's the best price we've seen for these headphones in any condition. It includes a 2-year Allstate warranty. Buy Now at eBay
Features: leather cushions ambient sound mode HD voice built-in mic up to 30-hour battery life Model: WH-1000XM5/B
URL: https://www.dealnews.com/products/Sony-Electronics/Certified-Refurb-Sony-WH-1000-XM5-Wireless-Headphones/351065.html?iref=rss-c142


In [6]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
Deals:
    
"""

USER_PROMPT_SUFFIX = """\n\nStrictly respond in JSON and include exactly 5 deals, no more.
Your response must follow this format:

{"deals": [
    {
        "product_description": "Your summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    {
        "product_description": "Your summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 210.30,
        "url": "the url as provided"
    },
    ...
]}
"""

In [8]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [9]:
user_prompt = make_user_prompt(scraped_deals)
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
Deals:
    
Title: Open-Box Bose QuietComfort 35 II Wireless Headphones for $105 + free shipping
Details: Apply coupon code "STARTFRESH" to drop the price to the second best we've seen in any condition. Buy Now at eBay
Features: foldable built-in microphone acoustic noise cancelling up to 20-hour battery life Model: 789564-0010 UPC: 820148577697 UPC: 017

In [10]:
# Create the user prompt that reflects the deals we have scraped

user_prompt = make_user_prompt(scraped_deals)

# Call OpenAI using their Python client. Request a response in JSON and stream it back.

stream = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
  ],
    max_tokens=2000,
    stream=True,
    response_format={"type": "json_object"}
)

response = ""
for chunk in stream:
    chunk_text = chunk.choices[0].delta.content or ''
    response += chunk_text
    print(chunk_text, end="", flush=True)

{
    "deals": [
        {
            "product_description": "The Bose QuietComfort 35 II Wireless Headphones provide an unparalleled listening experience with advanced acoustic noise cancelling technology. Designed for comfort, these foldable headphones include a built-in microphone and support for clear voice calls. With up to 20 hours of battery life, you can enjoy your music or podcasts throughout the day without interruption. The model features superior sound quality that is perfect for both casual listening and immersive audio experiences.",
            "price": 105,
            "url": "https://www.dealnews.com/products/Bose/Bose-Quiet-Comfort-35-II-Wireless-Headphones/104701.html?iref=rss-c142"
        },
        {
            "product_description": "The Sony WH-1000XM5 Wireless Headphones are a top-tier choice for audiophiles seeking high-quality sound and comfort. These headphones feature plush leather cushions and an ambient sound mode, allowing you to enjoy music while stay

In [11]:
import json
json.loads(response)

{'deals': [{'product_description': 'The Bose QuietComfort 35 II Wireless Headphones provide an unparalleled listening experience with advanced acoustic noise cancelling technology. Designed for comfort, these foldable headphones include a built-in microphone and support for clear voice calls. With up to 20 hours of battery life, you can enjoy your music or podcasts throughout the day without interruption. The model features superior sound quality that is perfect for both casual listening and immersive audio experiences.',
   'price': 105,
   'url': 'https://www.dealnews.com/products/Bose/Bose-Quiet-Comfort-35-II-Wireless-Headphones/104701.html?iref=rss-c142'},
  {'product_description': 'The Sony WH-1000XM5 Wireless Headphones are a top-tier choice for audiophiles seeking high-quality sound and comfort. These headphones feature plush leather cushions and an ambient sound mode, allowing you to enjoy music while staying aware of your surroundings. With a built-in microphone and HD voice t

# Putting this into an "Agent"

I've packaged this code into a class called `ScannerAgent`

There are various Agent frameworks that add an abstraction layer, but in our case it's easy just to write the code directly.

In [12]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [13]:
from agents.scanner_agent import ScannerAgent

scanner = ScannerAgent()
print(scanner.scan())

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 15 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Output
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


deals=[Deal(product_description='The Bose QuietComfort 35 II Wireless Headphones are designed for comfort and immersive sound. With acoustic noise cancelling technology, these headphones effectively block out ambient noise, allowing for an uninterrupted listening experience. They feature a foldable design with a built-in microphone for easy calls and voice commands, and offer up to 20 hours of battery life for extended use. Ideal for travelers or anyone seeking high-quality audio in a portable device.', price=105.0, url='https://www.dealnews.com/products/Bose/Bose-Quiet-Comfort-35-II-Wireless-Headphones/104701.html?iref=rss-c142'), Deal(product_description='The Sony WH-1000XM5 Wireless Headphones boast premium features, including leather cushions for comfort and ambient sound mode that allows for a seamless blend of outside sounds while listening. With HD voice technology and a built-in mic, these headphones are perfect for hands-free calls. They promise remarkable sound quality and up